![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population. Explore the `students` data using PostgreSQL to find out if this is true and see if the length of stay is a contributing factor.

Here is a data description of the fields you may find helpful. The full dataset is in one table with 50 fields and, according to the survey, 268 records. Each row is a student.

| Field Name    | Description                                      | 
| ------------- | ------------------------------------------------ |
| inter_dom     | Types of students                                |
| japanese_cate | Japanese language proficiency                    | 
| english_cate  | English language proficiency                     |
| academic      | Current academic level                           | 
| age           | Current age of student                           |
| stay          | Current length of stay in years                  |
| todep         | Total score of depression (PHQ-9 test)           |
| tosc          | Total score of social connectedness (SCS test)   |
| toas          | Total score of Acculturative Stress (ASISS test) |

## Exploration
Let's have a look at the data.

In [22]:
SELECT inter_dom, japanese_cate, english_cate, academic, age, stay, todep, tosc, toas
FROM students


,inter_dom,japanese_cate,english_cate,academic,age,stay,todep,tosc,toas
0,Inter,Average,High,Grad,24.0,5.0,0.0,34.0,91.0
1,Inter,High,High,Grad,28.0,1.0,2.0,48.0,39.0
2,Inter,High,High,Grad,25.0,6.0,2.0,41.0,51.0
3,Inter,Low,Average,Grad,29.0,1.0,3.0,37.0,75.0
4,Inter,Low,Average,Grad,28.0,1.0,3.0,37.0,82.0
...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,NaN,NaN
282,,,,,NaN,NaN,NaN,NaN,NaN
283,,,,,NaN,NaN,NaN,NaN,NaN
284,,,,,NaN,NaN,NaN,NaN,NaN


## How many International students are there?

In [23]:

SELECT inter_dom, COUNT(*) as count
FROM students
GROUP BY inter_dom;


,inter_dom,count
0,Dom,67
1,,18
2,Inter,201


## Avg age of students

In [24]:
SELECT inter_dom, round(AVG(age),1) as average_age
FROM students
GROUP BY inter_dom;

,inter_dom,average_age
0,Dom,20.4
1,,NaN
2,Inter,21.0


Very similiar ages.

## Language proficiency

Let's explore how good the foreign students Japanese is. Perhaps a lack of local language is contributing to their mental health form.

In [25]:
SELECT english_cate, num_students
FROM (SELECT english_cate, COUNT(*) as num_students
      FROM students
      WHERE inter_dom = 'Inter'
      GROUP BY english_cate) as student_counts


,english_cate,num_students
0,Average,41
1,High,153
2,Low,7


As you would expect, language students in Japan have very good English. 
Perhaps not having any Japanese is an issue?

In [26]:
SELECT japanese_cate, num_students
FROM (SELECT japanese_cate, COUNT(*) as num_students
      FROM students
      WHERE inter_dom = 'Inter'
      GROUP BY japanese_cate) as student_counts

,japanese_cate,num_students
0,Average,85
1,High,25
2,Low,91


While the majority of foreign students in Japan demonstrated a strong understanding of the Japanese language, 45% (91/201) showed lower levels of proficiency. Further research is necessary to determine the extent to which these students may require additional language support.

## Depression tests

Now we should look for a summary about the ranges of well-being scores within the sample.

In [27]:
SELECT stay,
        min(todep) as min_phq,
        min(tosc) as min_scs,
        min(toas) as min_as
FROM students
WHERE inter_dom = 'Inter'
Group by stay
ORDER BY stay DESC

,stay,min_phq,min_scs,min_as
0,10,13,32,50
1,8,10,44,65
2,7,4,48,45
3,6,2,35,42
4,5,0,34,91
5,4,0,17,36
6,3,0,13,36
7,2,0,11,36
8,1,0,11,36


The longer you stay the lower the minimum score. This means the mean score is being dragged lower due to lower score as time goes by. This could indicate a strong correlation between stay and wellbeing. Let's explore the max values next.

In [28]:
SELECT  stay,
        max(todep) as max_phq,
        max(tosc) as max_scs,
        max(toas) as max_as
FROM students
WHERE inter_dom = 'Inter'
Group by stay
ORDER BY stay DESC

,stay,max_phq,max_scs,max_as
0,10,13,32,50
1,8,10,44,65
2,7,4,48,45
3,6,10,41,83
4,5,0,34,91
5,4,14,48,129
6,3,24,48,133
7,2,21,48,127
8,1,25,48,145


The longer you stay the max valuescores in the tests go up. Again, this means that as long as people stay there the high scores decrease. People are feeling more depressed as time goes by.

In [29]:
SELECT stay,
       ROUND(AVG(todep), 2) as average_phq,
       ROUND(AVG(tosc), 2) as average_scs,
       ROUND(AVG(toas), 2) as average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC


,stay,average_phq,average_scs,average_as
0,10,13.00,32.00,50.00
1,8,10.00,44.00,65.00
2,7,4.00,48.00,45.00
3,6,6.00,38.00,58.67
4,5,0.00,34.00,91.00
5,4,8.57,33.93,87.71
6,3,9.09,37.13,78.00
7,2,8.28,37.08,77.67
8,1,7.48,38.11,72.80


The results indicate that, on average, international students who have been studying for longer lengths of time tend to have lower average diagnostic scores. For example, students who have been studying for 7 years or more have lower average scores for all three tests compared to students who have been studying for shorter lengths of time.

It is important to note that these results are only a general trend and may not apply to all international students. Individual students may have different experiences and scores that do not follow this trend. It would be necessary to do further analysis to understand the factors that contribute to these trends and how they vary among different groups of students.

## Conclusion
In this analysis, we explored the impact of the length of stay on the diagnostic scores of international students. We found that, on average, international students who have been studying for longer lengths of time tend to have lower diagnostic scores compared to students who have been studying for shorter lengths of time. This trend was observed for all three diagnostic tests: the depression (PHQ-9) test, the social connectedness (SCS) test, and the acculturative stress (ASISS) test.

Additionally, we found that the minimum and maximum diagnostic scores for international students also tend to decrease with longer lengths of stay. This suggests that the range of scores for international students may also be affected by the length of stay.

While these results provide insights into the relationship between the length of stay and diagnostic scores for international students, it is important to note that these results are only a general trend and may not apply to all international students. Individual students may have different experiences and scores that do not follow this trend. Further analysis is needed to understand the factors that contribute to these trends and how they vary among different groups of students.